In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
import csv

In [3]:
sc = pyspark.SparkContext(appName="Titanic")

In [4]:
#Cargamos el archivo csv y creamos un RDD con esta información
#raw: RDD de listas de cadenas de texto
#Primero leemos el archivo, dividimos el CSV en listas
#Listas, eliminamos la cabecera
raw = (sc.textFile("C:\\Big Data\\titanic.csv") \
    .map(lambda s: list (csv.reader([s]))[0]) \
    .filter(lambda l: l[0] !="survived"))
print("Lista totales: ", raw.count())

Lista totales:  891


In [5]:
raw.collect()

[['0',
  '3',
  'male',
  '22.0',
  '1',
  '0',
  '7.25',
  'S',
  'Third',
  'man',
  'True',
  '',
  'Southampton',
  'no',
  'False'],
 ['1',
  '1',
  'female',
  '38.0',
  '1',
  '0',
  '71.2833',
  'C',
  'First',
  'woman',
  'False',
  'C',
  'Cherbourg',
  'yes',
  'False'],
 ['1',
  '3',
  'female',
  '26.0',
  '0',
  '0',
  '7.925',
  'S',
  'Third',
  'woman',
  'False',
  '',
  'Southampton',
  'yes',
  'True'],
 ['1',
  '1',
  'female',
  '35.0',
  '1',
  '0',
  '53.1',
  'S',
  'First',
  'woman',
  'False',
  'C',
  'Southampton',
  'yes',
  'False'],
 ['0',
  '3',
  'male',
  '35.0',
  '0',
  '0',
  '8.05',
  'S',
  'Third',
  'man',
  'True',
  '',
  'Southampton',
  'no',
  'True'],
 ['0',
  '3',
  'male',
  '',
  '0',
  '0',
  '8.4583',
  'Q',
  'Third',
  'man',
  'True',
  '',
  'Queenstown',
  'no',
  'True'],
 ['0',
  '1',
  'male',
  '54.0',
  '0',
  '0',
  '51.8625',
  'S',
  'First',
  'man',
  'True',
  'E',
  'Southampton',
  'no',
  'True'],
 ['0',
  '3',
 

In [6]:
#Funciones
#Se verifica que las listas tenga información
def complete(l):
    for i in [0,1,2,3,4,5,6,7]: #Unicmaente comprobamos que tenga informacion
        if l[i]=='':
            return False
    return True
#Transforma la lista de cadenas de texto en diccionarios.
#segun la columna
#No se toman en cuenta las demas columnas: class, who , adult_male, deck, embark_town, alive y alone
def proyect_and_parse(l):
    return{"survived":int(l[0]), "pclass":int(l[1]),"sex":l[2],"age":float(l[3]),"sibsp":int(l[4]),"parch":int(l[5]),"fare":float(l[6]),"embarked":l[7]}

In [7]:
#RDD de diccionarios con valores válidos
non_null = raw.filter(complete)
r1 = (non_null.map(proyect_and_parse))
print("Listas completas: ", r1.count())

Listas completas:  712


In [8]:
#Funcion que acepta dos diccionarios y combina cada clave
#usando la funcion f
#Suponemos que ambos diccionarios tiene la mismas claves
def combine_dicts(f, d1, d2):
    res = dict()
    for k in d1.keys():
        res[k] = f(d1[k], d2[k])
    return res
#Dict que determina los valores maximos para cada columna
maximos = r1.reduce(lambda d1, d2: combine_dicts(max, d1, d2))
#Dict que determina valores minimos para cada columna
minimos = r1.reduce(lambda d1, d2: combine_dicts(min, d1,d2))

print("Maximos:", maximos)
print("Minimos:", minimos)

Maximos: {'survived': 1, 'pclass': 3, 'sex': 'male', 'age': 80.0, 'sibsp': 5, 'parch': 6, 'fare': 512.3292, 'embarked': 'S'}
Minimos: {'survived': 0, 'pclass': 1, 'sex': 'female', 'age': 0.42, 'sibsp': 0, 'parch': 0, 'fare': 0.0, 'embarked': 'C'}


In [9]:
#Funcion que procesa todas las claves en 'keys' de un diccionario y las 
#normaliza usando los diccionarios de valores máximos 'maxs' y minimos'minis'
#El resto de entradas se queda inalterado
def normalize(d, keys, maxs, mins):
    res = dict()
    for(k,v) in d.items():
        if k in keys:
            res[k] = (v-mins[k]) / (maxs[k]-mins[k])
        else:
            res[k] = v
    return res
#normalize : realiza una reescala de valores numericos
#Software, SW, soft = software, numeros, num #, No. = Numeros
normalized = r1.map(lambda d: normalize(d,['age','sibsp','parch','fare'], maximos, minimos))
print("Primera entrada:\n",r1.first())
print("Primeras entrada normalizada:\n",normalized.first(),'\n')

Primera entrada:
 {'survived': 0, 'pclass': 3, 'sex': 'male', 'age': 22.0, 'sibsp': 1, 'parch': 0, 'fare': 7.25, 'embarked': 'S'}
Primeras entrada normalizada:
 {'survived': 0, 'pclass': 3, 'sex': 'male', 'age': 0.2711736617240513, 'sibsp': 0.2, 'parch': 0.0, 'fare': 0.014151057562208049, 'embarked': 'S'} 



In [10]:
#Funcion que actualiza el conjunto de valores unicos con el valor que contiene el diccionar en la columna 'key'
def update_set_dicts(s, d, key):
    s.add(d[key])#Spark permite modificar el primer parametro y devolverlo
    return s
#Diccionario de conjunto de valores unicos para la columna 'sex'
#set(): crear un diccionario vacio, no permite datos repetidos
sex_values = normalized.aggregate(set(),
                                        lambda s, d: update_set_dicts(s,d,'sex'),
                                        lambda s1, s2: s1.union(s2))
#Diccionario de conjunto de valores unicos para la columna 'embarked'

embarked_values = normalized.aggregate(set(),
                                      lambda s, d: update_set_dicts(s, d, 'embarked'),
                                      lambda s1, s2: s1.union(s2))
#Realizamos los prints

print("Valores unicos de la columna sex:", sex_values)
print("Valores unicos de la coolumna embarked", embarked_values)

Valores unicos de la columna sex: {'male', 'female'}
Valores unicos de la coolumna embarked {'C', 'S', 'Q'}


In [11]:
#Funcion que acepta un objeto iterable(lista, conjunto, ndarray....)
#de valores unicos y 
#devuelve un diccionario para traducir valores -- numerso.
#los numeros naturales estan en el rango [0, len(it)]
def values_to_mapping(it):
    values = sorted(it)
    size = len(values)
    return{k:v for (k,v) in zip(values, range(size))} #recibe como parametros objetos iterables del mismo tamaño
    #resultado : (k:v)
#Construye los diccionarios traductores a parte de los cojunto de valores
sex_mapping = values_to_mapping(sex_values)
embarked_mapping = values_to_mapping(embarked_values)
print("Mapping para columna sex: ", sex_mapping)
print("Mapping para columna embarked", embarked_mapping,'\n')

Mapping para columna sex:  {'female': 0, 'male': 1}
Mapping para columna embarked {'C': 0, 'Q': 1, 'S': 2} 



In [12]:
#Funcion que dada una diccionario y una clave cambio su valor por su traduccion segun 'mapping'
def dict_translate(d, key, mapping):
    d[key] = mapping[d[key]]
    return d
#Traduce las entradas 'sex' y 'embarked' de los diccionarios por sus valores traducidos
titanic = (normalized.map(lambda d: dict_translate(d, 'sex',sex_mapping)).map(lambda d: dict_translate(d,'embarked',embarked_mapping)))
#Entradas preprocesadas: una aplicadas varias transformaciones los datos est
#con algun algorimot del ML
print("Primera entrada preprocesada:",titanic.first())

Primera entrada preprocesada: {'survived': 0, 'pclass': 3, 'sex': 1, 'age': 0.2711736617240513, 'sibsp': 0.2, 'parch': 0.0, 'fare': 0.014151057562208049, 'embarked': 2}


In [13]:
sc.stop()